## Setup

In [ ]:
# packages which are needed, install in current environment
# python >= 3.8
%pip install colorama
%pip install simpleaudio
%pip install python-rtmidi
%pip install mido
%pip install pynput

In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
from utils.log import Log
log = Log(Log.LEVEL_INF, name='notebook')

## Hello World

In [6]:
import midi
import canvas

class Hello(midi.Module):
  def note(self, event: midi.Note):
    self.send(event) # just pass event further
    
    self.redraw(note=event.note)
    
  def redraw(self, note=None):
    # write 'Hello' to the display
    canvas.write(f'Hello {note if note != None else ""}')

In [7]:
from utils.midi_tester import Tester, TesterOutput

t = Tester(Hello(), out=TesterOutput.AUDIO)
t.start()

This process is not trusted! Input event monitoring will not be possible until it is added to accessibility clients.



[IMP] canvas: Hello 

[INF] midi_tester: Starting keyboard listener, stop by pressing ESC or ENTER
[INF] midi_tester: Use LEFT and RIGHT to control the encoder

[IMP] canvas: Hello 60
[IMP] canvas: Hello 60
[IMP] canvas: Hello 60
[IMP] canvas: Hello 60
[IMP] canvas: Hello 60
[IMP] canvas: Hello 60

[INF] midi_tester: 
[INF] midi_tester: Keyboard listener stopped
[INF] midi_tester: Audio stopped
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/rein/Library/miniconda3/envs/midisim/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/y7/str5z30s4lq1hd9vvv9c0sk40000gn/T/ipykernel_31418/576364577.py", line 4, in <module>
    t.start()
  File "/Users/rein/Documents/Prosjekter/midi_keyboard/scipt_simulator/midi_tester.py", line 197, in start
    listener.join()
  File "/Users/rein/Library/miniconda3/envs/midisim/lib/python3.11/site-packages/pynput/_util/__init__.py", line 269, in join
    super(AbstractListener, self).join(*args)
  File "/Users/rein/Library/miniconda3/envs/midisim/lib/python3.11/threading.py", line 1112, in join
    self._wait_for_tstate_lock()
  File "/Users/rein/Library/miniconda3/envs/midisim/lib/python3.11/threading.py", line 1132, in _wait_for_tstate_lock
    if lock.acquire(block, timeout):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt

During 

## Pulse

In [4]:
import midi
import canvas

class Pulse(midi.Module):
  
  TICKS_PER_NOTE_OPTIONS = [2, 4, 8, 12]
  
  def __init__(self):
    self.reset()
    
  def reset(self):
    self.playing = []
    self.ticks = 0
    self.next_press = True
    self.ticks_per_note_select = 2
  
  def note(self, k):
    if k.pressed:
      self.playing.append(k)
    else:
      if k in self.playing:
        if not self.next_press:
          self.send(k)
        self.playing.remove(k)
    
  def tick(self):    
    self.ticks += 1
    if self.ticks < self.TICKS_PER_NOTE_OPTIONS[self.ticks_per_note_select]:
      return
    
    self.ticks = 0

    if self.next_press:
      for k in self.playing:
        self.send(k)
    else:
      for k in self.playing:
        self.send(k.copy_with(state=midi.State.NOTE_OFF))
        
    self.next_press = not self.next_press
    
  def enc(self, inc):
    """Physical rotary encoder events. Changes the arpeggio speed"""
    
    # selects the speed and bound checks
    self.ticks_per_note_select = min(max(self.ticks_per_note_select + inc, 0), len(self.TICKS_PER_NOTE_OPTIONS) - 1)

    # update screen graphics
    self.redraw()
        
  def redraw(self):
    ticks_per_note = self.TICKS_PER_NOTE_OPTIONS[self.ticks_per_note_select]
    canvas.write(f'*PLS: 1/{ticks_per_note}')

In [ ]:
from utils.midi_tester import Tester, TesterOutput

log.set_level(log.LEVEL_INF)

t = Tester(Pulse(), bpm=120, out=TesterOutput.MIDI)
t.start()

## Arpeggio creation

Firsty, we will import the core libraries which is needed when creating a new midi scetch

In [12]:
import midi
import canvas

class Arpeggio(midi.Module):
  TICKS_PER_NOTE_OPTIONS = [1, 4, 8, 12]
  
  def __init__(self):
    self.reset()

  def reset(self):
    """Sets the scetch to its initial state"""
    self.tick_count = 0
    self.ticks_per_note_select = 2
    self.last_played_note = None
    self.current_note_idx = 0
    self.notes_active = []
 
  @property
  def ticks_per_note(self):
    return self.TICKS_PER_NOTE_OPTIONS[self.ticks_per_note_select]
 
  def note(self, event: midi.Note):
    """Processes note events. Adds or removes from arpeggio"""
    if event.pressed:
      self.notes_active.append(event)
    else:
      self.notes_active.remove(event)

  def enc(self, inc):
    """Physical rotary encoder events. Changes the arpeggio speed"""
    # selects the arpeggio speed and bound checks
    self.ticks_per_note_select = min(max(self.ticks_per_note_select + inc, 0), len(self.TICKS_PER_NOTE_OPTIONS) - 1)

    # update display graphics
    self.redraw()

  def tick(self):    
    self.tick_count += 1
    if self.tick_count < self.ticks_per_note:
      return
    self.tick_count = 0
                
    # stop last played note
    if self.last_played_note != None:
      event = self.last_played_note.copy_with(state=midi.State.NOTE_OFF)
      self.send(event)
      self.last_played_note = None
      
    # play new note
    if len(self.notes_active) > 0:
      if self.current_note_idx >= len(self.notes_active):
        self.current_note_idx = 0
      event = self.notes_active[self.current_note_idx]
      self.send(event)
      self.last_played_note = event
      self.current_note_idx += 1

  def redraw(self):
    canvas.write(f'*ARP:1/{self.ticks_per_note}')

## Test

In [13]:
import utils.midi_tester
from utils.midi_tester import Tester, TesterOutput

# midi_tester.log.set_level(midi_tester.log.LEVEL_DBG)

t = Tester(Arpeggio(), bpm=120, out=TesterOutput.MIDI)
t.start()

This process is not trusted! Input event monitoring will not be possible until it is added to accessibility clients.



[IMP] canvas: *ARP:1/8

[INF] midi_tester: Starting keyboard listener, stop by pressing ESC or ENTER
[INF] midi_tester: Use LEFT and RIGHT to control the encoder
[INF] midi_tester: 
[INF] midi_tester: Keyboard listener stopped
[INF] midi_tester: MIDI out closed
